### Import libraries

In [1]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Label, LabelSet, Title, Range1d
from bokeh.transform import dodge, linear_cmap
from bokeh.models.glyphs import Text

output_notebook()

Loading BokehJS ...

### Our objetive

<img src="../../images/figure_4.png" width="50%" height="50%">

#### Preprocessing 

In [2]:
dataset = pd.read_csv('../../data/data_figure_1.csv')
dataset.shape

(5, 5)

In [3]:
dataset.head()

,interest,before,after,formatted_before,formatted_after
0,Bored,11,12,11%,12%
1,Not great,5,6,5%,6%
2,OK,40,14,40%,14%
3,Kind of interested,25,30,25%,30%
4,Excited,19,38,19%,38%


In [4]:
dataset_h = dataset[['interest', 'before', 'after']].set_index('interest').T.rename_axis('When').sort_values('When', ascending=True).reset_index()

In [5]:
dataset_h['bored_formatted'] = dataset_h['Bored'].astype(str) + '%'
dataset_h['notgreat_formatted'] = dataset_h['Not great'].astype(str) + '%'
dataset_h['ok_formatted'] = dataset_h['OK'].astype(str) + '%'
dataset_h['kind_formatted'] = dataset_h['Kind of interested'].astype(str) + '%'
dataset_h['excited_formatted'] = dataset_h['Excited'].astype(str) + '%'

In [6]:
dataset_h

interest,When,Bored,Not great,OK,Kind of interested,Excited,bored_formatted,notgreat_formatted,ok_formatted,kind_formatted,excited_formatted
0,after,12,6,14,30,38,12%,6%,14%,30%,38%
1,before,11,5,40,25,19,11%,5%,40%,25%,19%


In [7]:
dataset_h.sort_values('When', ascending=False, inplace=True)
dataset_h['When'] = dataset_h['When'].str.upper()

In [8]:
# Bokeh doesn't allow you to add off-axis text. This is a little trick to do it
new_row = {'When':''}
dataset_h = dataset_h.append(new_row, ignore_index=True)
new_row = {'When':' '}
dataset_h = dataset_h.append(new_row, ignore_index=True)

In [9]:
categories = dataset_h.set_index('When').columns.tolist()[:5]
labels = dataset_h.set_index('When').columns.tolist()[5:]

#### Colors 

In [10]:
GRAY_1, GRAY_2, GRAY_3 = '#A6A6A5', '#929497', '#231F20'
GRAY_4, GRAY_5, GRAY_6 = '#838383', '#555655', '#828282'
GRAY_7, GRAY_8, GRAY_9 = '#646369', '#333333', '#9c9c9c'
GRAY_10 = '#76787B'
BLUE_1, BLUE_2, BLUE_3 = '#94B2D7', '#4A81BF', '#174A7E'
ORANGE_1 = '#F79747'

In [11]:
colors = [GRAY_1, GRAY_1, ORANGE_1, BLUE_2, BLUE_2]

#### Font 

In [12]:
FONT = 'Arial'

#### Plot

In [107]:
# Set the source of the plot
source = ColumnDataSource(dataset_h)
source_after = ColumnDataSource(dataset_h.loc[dataset_h['When'] == 'AFTER'])
source_before = ColumnDataSource(dataset_h.loc[dataset_h['When'] == 'BEFORE'])


# Create the figure
p = figure(x_range=list(dataset_h['When']),
           y_range=(0, 45), 
           plot_height=450, 
           plot_width=1000, 
           title='Pilot program was a success' + ' ' * 50, 
           toolbar_location='above')


# Add bars to the figure
for i, category in enumerate(categories):
    p.line(x='When', 
           y=category,
           line_width=5,
           color=colors[i],
           source=source)

    p.circle(x='When', 
             y=category, 
             size=10,
             line_width=5,         
             color=colors[i],
             fill_color=colors[i],
             source=source)
    
    p.add_layout(LabelSet(x='When', 
                      y=category, 
                      text=labels[i],
                      text_align='center',
                      x_offset=-30,
                      y_offset=-10,
                      background_fill_alpha=1,
                      text_font=FONT,
                      text_font_size='15pt',
                      text_color=colors[i], 
                      text_baseline='bottom', 
                      source=source_before))

    
    p.add_layout(LabelSet(x='When', 
                      y=category, 
                      text=labels[i], 
                      text_align='center',
                      x_offset=30,
                      y_offset=-10,
                      background_fill_alpha=1,
                      text_font=FONT,
                      text_font_size='15pt',
                      text_color=colors[i], 
                      text_baseline='bottom', 
                      source=source_after))


# Add subtitles
p.add_layout(Title(text='How do you feel about science?', 
                   align="left",
                   offset=70,
                   text_font_size='14pt', 
                   text_color=GRAY_3, 
                   text_font_style='normal'), 
             "above")


# Add footing legend
p.add_layout(Title(text='Based on survey of 100 students conducted before and after pilot program (100% response rate on both surveys).', 
                   align="left",
                   offset=70,
                   text_font_size='10pt', 
                   text_color=GRAY_7), 
             "below")

# Add text annotations
# This is a weakness of bokeh, you cannot format a single word within a text. We have to do it separately.
p.text(x=2, 
       y=33,
       x_offset=45,
       y_offset=-55,
       text=['BEFORE program, the'],
       text_font_size='18pt', 
       text_color=GRAY_4)

p.text(x=2, 
       y=33,
       x_offset=45,
       y_offset=13,
       text=['majority of children felt \njust Ok'],
       text_font_size='18pt', 
       text_color=ORANGE_1)

p.text(x=2, 
       y=33,
       x_offset=45,
       y_offset=13,
       text=['\t' * 13 + 'about science.'],
       text_font_size='18pt', 
       text_color=GRAY_4)

p.text(x=2, 
       y=20,
       x_offset=45,
       y_offset=-25,
       text=['AFTER program,'],
       text_font_size='18pt', 
       text_color=GRAY_4)

p.text(x=2, 
       y=20,
       x_offset=45,
       y_offset=70,
       text=['more children where \nKind of interested & \nExcited'],
       text_font_size='18pt', 
       text_color=BLUE_2)

p.text(x=2, 
       y=20,
       x_offset=55,
       y_offset=70,
       text=['\t' * 12 + 'about science.'],
       text_font_size='18pt', 
       text_color=GRAY_4)


p.text(x=2, 
       y=38,
       x_offset=-55,
       y_offset=10,
       text=['Excited'],
       text_font_size='15pt', 
       text_color=BLUE_2)

p.text(x=2, 
       y=30,
       x_offset=-55,
       y_offset=40,
       text=['Kind of \ninterest'],
       text_font_size='15pt', 
       text_color=BLUE_2)

p.text(x=2, 
       y=14,
       x_offset=-55,
       y_offset=10,
       text=['OK'],
       text_font_size='15pt', 
       text_color=ORANGE_1)

p.text(x=2, 
       y=12,
       x_offset=-55,
       y_offset=13,
       text=['Bored'],
       text_font_size='15pt', 
       text_color=GRAY_1)

p.text(x=2, 
       y=6,
       x_offset=-55,
       y_offset=10,
       text=['Not great'],
       text_font_size='15pt', 
       text_color=GRAY_1)
    
    
    
# Elements attributes
p.segment(x0=0.5, 
          y0=0, 
          x1=1.5,
          y1=0, 
          color=GRAY_2, 
          line_width=1)


# Modify title attributes
p.title.text_color = 'white'
p.title.background_fill_color = GRAY_2
p.title.text_font = FONT
p.title.text_font_size = '20pt'
p.title.text_font_style = 'normal'
p.title.offset = 70


# Modify X axis attributes
p.xaxis.axis_line_color = None
p.xgrid.grid_line_color = None
p.xaxis.major_tick_line_color = GRAY_2
p.xaxis.major_label_text_color = GRAY_2
p.xaxis.major_label_text_font_size = '11pt'
p.xaxis.major_label_standoff = 10
p.xaxis.major_tick_in = 0
p.xaxis.major_tick_out = 5
p.xaxis.minor_tick_line_color = None


# Modify Y axis attributes
p.ygrid.visible = False
p.yaxis.axis_line_color = None
p.yaxis.major_tick_line_color = None
p.yaxis.major_label_text_color = None

p.yaxis.minor_tick_line_color = None

# Convert the figure to png
p.outline_line_color = None
p.background_fill_color = None
p.border_fill_color = None


show(p)